In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
dbutils.widgets.text("storage-account", "adslproyecto01")
dbutils.widgets.text("esquema_source", "silver")
dbutils.widgets.text("esquema_sink", "golden")

In [0]:
##Definicion de parametros
storage_account = dbutils.widgets.get("storage-account")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_sink = dbutils.widgets.get("esquema_sink")


In [0]:
df_reporte_b3 = spark.read \
    .option("header", True) \
    .csv(f"abfss://{esquema_source}@{storage_account}.dfs.core.windows.net/reporte_b3_silver/")

In [0]:
df_reporte_b3_transformed = df_reporte_b3 \
                    .groupBy(
                        col("año"),
                        col("mes"),
                        col("cod_oficina"),
                        col("desc_tipo_solicitud"),
                        col("desc_tramite"),col("servicio"), 
                        when(col("espera_mayor_15min") == "Si", "> 15min").otherwise("< 15min").alias("variable")
                        ).agg(count(col("cod_arribo")).alias("valor")) \
                    .orderBy(col("desc_tipo_solicitud").asc())

In [0]:
df_reporte_b3_transformed.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv(f"abfss://{esquema_sink}@{storage_account}.dfs.core.windows.net/reporte_b3_golden")                

In [0]:
scope = "accessScopeforASQLDB"
key = "DBPassword"

In [0]:
jdbcHostname = "serverproyecto01.database.windows.net"
jdbcDatabase = "bdproyecto01"
jdbcPort = 1433
jdbcUsername = "sqladmin"
jdbcPassword = dbutils.secrets.get(scope=f"{scope}", key=f"{key}")

'[REDACTED]'

In [0]:
# URL JDBC
jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"

In [0]:
df_reporte_b3_transformed.write \
    .format("jdbc") \
    .option("url", jdbcUrl) \
    .option("dbtable", "Reporte_A2") \
    .option("user", jdbcUsername) \
    .option("password", jdbcPassword) \
    .mode("overwrite") \
    .save()